In [1]:
import torch
import torch.nn.functional as F
import requests
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig
# from datasets import load_dataset

import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import logging
logger = logging.getLogger(__name__)

func_to_enable_grad = '_sample'
setattr(LlavaForConditionalGeneration, func_to_enable_grad, torch.enable_grad(getattr(LlavaForConditionalGeneration, func_to_enable_grad)))

C:\Users\PRYth\anaconda3\envs\fyp\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\PRYth\anaconda3\envs\fyp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "llava-hf/llava-1.5-7b-hf"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    offload_state_dict=True
)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
    quantization_config=quant_config,
    attn_implementation = "eager"
).to(0)

#--------------------------------------------------
model.vision_tower.config.output_attentions = True

# set hooks to get attention weights
model.enc_attn_weights = []
#outputs: attn_output, attn_weights, past_key_value
def forward_hook(module, inputs, output): 
    if output[1] is None:
        logger.error(
            ("Attention weights were not returned for the encoder. "
            "To enable, set output_attentions=True in the forward pass of the model. ")
        )
        return output
    
    output[1].requires_grad_(True)
    output[1].retain_grad()
    model.enc_attn_weights.append(output[1].detach().cpu())
    return output

hooks_pre_encoder, hooks_encoder = [], []
for layer in model.language_model.layers:
    hook_encoder_layer = layer.self_attn.register_forward_hook(forward_hook)
    hooks_pre_encoder.append(hook_encoder_layer)

model.enc_attn_weights_vit = []

def forward_hook_image_processor(module, inputs, output): 
    if output[1] is None:
        logger.warning(
            ("Attention weights were not returned for the vision model. "
             "Relevancy maps will not be calculated for the vision model. " 
             "To enable, set output_attentions=True in the forward pass of vision_tower. ")
        )
        return output

    output[1].requires_grad_(True)
    output[1].retain_grad()
    model.enc_attn_weights_vit.append(output[1])
    return output

hooks_pre_encoder_vit = []
for layer in model.vision_tower.vision_model.encoder.layers:
    hook_encoder_layer_vit = layer.self_attn.register_forward_hook(forward_hook_image_processor)
    hooks_pre_encoder_vit.append(hook_encoder_layer_vit)
#--------------------------------------------------

processor = AutoProcessor.from_pretrained(model_id)

if model.language_model.config.model_type == "gemma":
    eos_token_id = processor.tokenizer('<end_of_turn>', add_special_tokens=False).input_ids[0]
else:
    eos_token_id = processor.tokenizer.eos_token_id

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 3/3 [00:29<00:00,  9.70s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
tokens = torch.load("full_generation.pt").tolist()
# Decode each token individually
decoded_tokens = [processor.decode([t]) for t in tokens]

# Optionally, print
for i, tok in enumerate(decoded_tokens):
    print(f"{i}: {repr(tok)}")

In [ ]:
file = r"saved/vit_attn"
import os
torch.save(tokens, os.path.join(file, "lmao.pt"))

In [3]:
conversation = [
    {
      "role": "user",
      "content": [
          {"type": "text", "text": "What is this animal?"},
          {"type": "image"},
        ],
    },
]
conv_step = conversation + [
    {"role": "assistant", "content": [{"type": "text", "text": "This animal is a dog"}]}
]
conv_step

[{'role': 'user',
  'content': [{'type': 'text', 'text': 'What is this animal?'},
   {'type': 'image'}]},
 {'role': 'assistant',
  'content': [{'type': 'text', 'text': 'This animal is a dog'}]}]

In [4]:
image_file = r"C:\Users\PRYth\OneDrive\Pictures\Screenshots\Screenshot 2025-09-24 103058.png"
raw_image = Image.open(image_file).convert("RGB")
prompt_appended = processor.apply_chat_template(conv_step, add_generation_prompt=False, return_tensors="pt")
inputs_appended = processor(images=raw_image, text=prompt_appended, return_tensors='pt').to(0, torch.float16)

output_appended = model(**inputs_appended,
                        use_cache=False,
                        output_attentions=True,
                        output_hidden_states=True,
                        return_dict_in_generate=True,
                        output_scores=True,
                        eos_token_id=eos_token_id)
print(len(model.enc_attn_weights))

32


In [21]:
decoded_input_tokens = [processor.decode([t]) for t in inputs_appended["input_ids"][0]]

# Optionally, print
for i, tok in enumerate(decoded_input_tokens):
    print(f"{i}: {repr(tok)}")

0: '<s>'
1: 'US'
2: 'ER'
3: ':'
4: ''
5: '<image>'
6: '<image>'
7: '<image>'
8: '<image>'
9: '<image>'
10: '<image>'
11: '<image>'
12: '<image>'
13: '<image>'
14: '<image>'
15: '<image>'
16: '<image>'
17: '<image>'
18: '<image>'
19: '<image>'
20: '<image>'
21: '<image>'
22: '<image>'
23: '<image>'
24: '<image>'
25: '<image>'
26: '<image>'
27: '<image>'
28: '<image>'
29: '<image>'
30: '<image>'
31: '<image>'
32: '<image>'
33: '<image>'
34: '<image>'
35: '<image>'
36: '<image>'
37: '<image>'
38: '<image>'
39: '<image>'
40: '<image>'
41: '<image>'
42: '<image>'
43: '<image>'
44: '<image>'
45: '<image>'
46: '<image>'
47: '<image>'
48: '<image>'
49: '<image>'
50: '<image>'
51: '<image>'
52: '<image>'
53: '<image>'
54: '<image>'
55: '<image>'
56: '<image>'
57: '<image>'
58: '<image>'
59: '<image>'
60: '<image>'
61: '<image>'
62: '<image>'
63: '<image>'
64: '<image>'
65: '<image>'
66: '<image>'
67: '<image>'
68: '<image>'
69: '<image>'
70: '<image>'
71: '<image>'
72: '<image>'
73: '<image>'
7

In [25]:
print(output_appended.logits.shape)
# Decode them
topk = torch.topk(output_appended.logits[:, -7], k=1, dim=-1)
for ids in topk.indices:
    print(processor.tokenizer.decode(ids))

torch.Size([1, 599, 32064])
This
